In [8]:
import os
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 불러옵니다.
# OPENAI_API_KEY가 .env 파일에 설정되어 있어야 합니다.
load_dotenv()

True

In [ ]:
# Pydantic 모델을 정의합니다.
# 이 모델은 LLM이 생성해야 할 출력의 구조를 명시합니다.
class NewsSummary(BaseModel):
    """뉴스 기사 요약 모델"""

    title: str = Field(description="뉴스 기사의 제목")
    subtitle: str = Field(description="뉴스 기사의 부제목")
    summary: str = Field(description="뉴스 기사의 핵심 내용을 100자 이내로 요약한 내용")

In [10]:
# PydanticOutputParser를 초기화합니다.
# 이 파서는 Pydantic 모델에 정의된 스키마에 따라 LLM의 출력을 파싱합니다.
parser = PydanticOutputParser(pydantic_object=NewsSummary)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "뉴스 기사 요약 모델", "properties": {"title": {"description": "뉴스 기사의 제목", "title": "Title", "type": "string"}, "subtitle": {"description": "뉴스 기사의 부제목", "title": "Subtitle", "type": "string"}, "summary": {"description": "뉴스 기사의 핵심 내용을 100자 이내로 요약한 내용", "title": "Summary", "type": "string"}}, "required": ["title", "subtitle", "summary"]}
```


In [ ]:
# 프롬프트 템플릿을 생성합니다.
# `format_instructions` 변수는 파서가 요구하는 출력 형식을 동적으로 삽입합니다.
prompt_template = """
당신은 뉴스 기사를 읽고 아래에 주어진 형식에 맞게 정보를 추출하여 요약하는 전문가입니다.
주어진 형식에 맞게 JSON을 출력해야 합니다.

{format_instructions}

다음은 요약할 뉴스 기사입니다:
{news_article}
"""

# PromptTemplate 객체를 생성하고, 필요한 변수(format_instructions, news_article)를 설정합니다.
prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["news_article"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# 예제 뉴스 기사입니다.
example_news_article = """
금어기 풀렸다…롯데마트, 살아있는 서해안 햇꽃게 판매 개시
부산 격포항·태안 신진도항 소재 어선과 사전 계약…역대 최대 물량 준비

유통입력 :2025/08/19 09:41    수정: 2025/08/19 10:20

김민아 기자

롯데마트는 오는 21일 올해 첫 어획한 ‘서해안 햇꽃게’를 선보인다고 19일 밝혔다. 꽃게는 산란기 보호를 위해 매년 6월 21일부터 8월 20일까지 금어기를 시행하고 있어, 이번에 판매하는 상품은 금어기 해제 당일 새벽 서해안에서 잡아올린 물량이다.

오는 27일까지는 ‘서해안 햇꽃게(100g·냉장·국산)’를 행사 카드 결제시 20% 할인해 992원에 판매한다.

롯데마트는 새벽에 어획한 햇꽃게를 산지 직송해, 당일 오후 살아있는 상태로 공급한다. 조업 직후 꽃게를 5℃ 이하 냉수에 담가 기절시키고 모래톱과 유사한 환경을 조성해 전국 매장에 직송하는 구조다.

물량도 역대 최대로 준비했다. 롯데마트는 꽃게 주산지인 부안 격포항과 태안 신진도항 소재의 선단 40여 척과 사전 계약을 체결해, 전년 대비 약 30% 확대된 공급망을 구축했다.
"""

# 모델을 초기화합니다.
# OpenAI API 키는 환경 변수에서 자동으로 로드됩니다.
model = ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

# 프롬프트, 모델, 파서를 체인으로 연결합니다.
chain = prompt | model | parser

# 체인을 실행하고 요약 결과를 얻습니다.
# `invoke` 메서드는 체인을 실행하여 최종 결과를 반환합니다.
# PydanticOutputParser 덕분에 결과는 NewsSummary 객체 형태로 반환됩니다.
summary_result = chain.invoke({"news_article": example_news_article})

# 결과 출력
print("--- 요약 결과 ---")
print(f"제목: {summary_result.title}")
print(f"부제목: {summary_result.subtitle}")
print(f"요약: {summary_result.summary}")
print("--- 객체 정보 ---")
print(f"타입: {type(summary_result)}")
print(f"내용: {summary_result}")

--- 요약 결과 ---
제목: 금어기 풀렸다…롯데마트, 살아있는 서해안 햇꽃게 판매 개시
부제목: 부산 격포항·태안 신진도항 소재 어선과 사전 계약…역대 최대 물량 준비
요약: 롯데마트가 금어기 해제 당일 서해안에서 잡은 햇꽃게를 21일부터 판매하며, 27일까지 할인 행사도 진행한다.
--- 객체 정보 ---
타입: <class '__main__.NewsSummary'>
내용: title='금어기 풀렸다…롯데마트, 살아있는 서해안 햇꽃게 판매 개시' subtitle='부산 격포항·태안 신진도항 소재 어선과 사전 계약…역대 최대 물량 준비' summary='롯데마트가 금어기 해제 당일 서해안에서 잡은 햇꽃게를 21일부터 판매하며, 27일까지 할인 행사도 진행한다.'
